## Streaming data from APIs

A module is a group of elements whose interactions can be contained, abstracted and hidden under a small interface compared to the interface that would be the result exposing their full domain.

The below function, `readAnswer` contains logic for consuming an asynrchronous stream of strings, and sending it to a consumer until there's no more strings to consume. Loops are one of the typical domains that can be contained and exposed through a small interface.

Another of the guiding principiles of this design is to expose the minimal interface of existing abstractions. That's the case of `MailboxProcessor<Message>`, which is used in `readAnswer` just an `unit -> Async<string option>` (type `ReadString`).

This has the advantage that pieces with substantial internal logic can be implemented in isolation and then assembled into a full program. The challenge then becomes to find how we can decompose our design into such self-contained modules.

Module for consuming a stream

In [20]:
type StopInsert = {insertWord: string -> unit; stop: unit -> unit}

type ReadString = unit -> Async<string option>

let readAnswer (read: ReadString) (si: StopInsert) =
    let rec loop ()=
        task {
            let! r = read()
            match r with
            | Some w -> 
                si.insertWord w
                return! loop ()
            | None -> 
                si.stop()
        }
    loop() |> Async.AwaitTask |> Async.Start
    

Module for defining stream


Another indicators of possible decomposition are:
- we need code to initialize values
- we have producers and consumers 

In [ ]:
#r "nuget: FSharp.Control.AsyncSeq"

open FSharp.Control

type Message = AnswerSegment of AsyncReplyChannel<string>

type Provider = MailboxProcessor<Message> -> Async<unit>
type GetProvider = unit -> AsyncSeq<string>

let readSegments (inbox: MailboxProcessor<Message>) (xs: AsyncSeq<string>) =
  xs
  |> AsyncSeq.takeWhileAsync (fun x ->
    async {
      let! msg = inbox.TryReceive()

      return
        match msg with
        | Some (AnswerSegment chan) ->
          chan.Reply x
          true
        | _ -> false
    })
  |> AsyncSeq.toListAsync
  |> Async.Ignore

let stream (g: GetProvider) =
  let mb = MailboxProcessor.Start (fun inbox -> g() |> readSegments inbox)
  fun () -> mb.PostAndTryAsyncReply(AnswerSegment, timeout = 1000)

Definining and consuming stream

In [22]:
let streamFlow (g: GetProvider) (si: StopInsert) = readAnswer (stream g) si

## Implementing `GetProvider`

In [23]:
open FSharp.Control

type Model = string
type Provider = string
type Prompt = string
type Key = string
type KeyEnvVar = string

type Active = {provider: Provider; model: Model}
type ProviderImpl = {models: Model list; answerer: Model -> Prompt -> AsyncSeq<string>}

type Conf = {active: Active; providers: Map<Provider, ProviderImpl>}

type ProviderModule = {implementation: Key -> ProviderImpl; keyVar: KeyEnvVar; provider: Provider}

let getenv s =
  System.Environment.GetEnvironmentVariable s |> Option.ofObj

let initProviders (xs: ProviderModule list) (_default: Provider) =
  let providers = 
    xs
    |> List.choose (fun pm ->
       getenv pm.keyVar |> Option.map (fun key -> 
         pm.provider, pm.implementation key
       ) 
    )
    |> Map.ofList
  let active = {provider = _default; model = providers[_default].models.Head}
  {active = active; providers = providers}

let getProvider (conf: unit -> Conf) (getPrompt: unit -> Prompt) () =
  let c = conf ()
  let prompt = getPrompt ()
  c.providers[c.active.provider].answerer c.active.model prompt
  

## Implementing `StopInsert`

In [ ]:
#r "nuget:GtkSharp, 3.24.24.95"
open Gtk

type AdjustWord =
  { chatDisplay: TextView
    adjustment: Adjustment }

let insertWord (b: Builder) : string -> unit =
  let adj = b.GetObject "text_adjustment" :?> Adjustment
  let chatDisplay = b.GetObject "chat_display" :?> TextView
  let f w =
    chatDisplay.Buffer.PlaceCursor chatDisplay.Buffer.EndIter
    chatDisplay.Buffer.InsertAtCursor w
    adj.Value <- adj.Upper

  f

let newStopInsert (builder: Builder) =
  let answerSpinner = builder.GetObject "answer_spinner" :?> Spinner

  { stop = answerSpinner.Stop
    insertWord = insertWord builder }